## Libraries

In [1]:
!pip install -q --upgrade wandb

In [2]:
!pip install gdown

In [3]:
# !gdown https://drive.google.com/file/d/15pp6g1dVh7kNXa-eXSMi_-1DTbuB2kIF/view?usp=sharing
# !gdown https://drive.google.com/file/d/1BYoR55Z9SAKKs9IQplu7ybG3p_osRDAH/view?usp=sharing
!gdown "https://drive.google.com/uc?id=15pp6g1dVh7kNXa-eXSMi_-1DTbuB2kIF&confirm=t"
!gdown "https://drive.google.com/uc?id=1BYoR55Z9SAKKs9IQplu7ybG3p_osRDAH&confirm=t"

Downloading...
From: https://drive.google.com/uc?id=15pp6g1dVh7kNXa-eXSMi_-1DTbuB2kIF&confirm=t
To: /kaggle/working/train_data.csv
100%|█████████████████████████████████████████| 725M/725M [00:04<00:00, 161MB/s]
Downloading...
From: https://drive.google.com/uc?id=1BYoR55Z9SAKKs9IQplu7ybG3p_osRDAH&confirm=t
To: /kaggle/working/test_data.csv
100%|███████████████████████████████████████| 7.03M/7.03M [00:00<00:00, 117MB/s]


In [2]:
!pip install mlxtend

  Using cached scikit_learn-1.3.2-cp38-cp38-win_amd64.whl.metadata (11 kB)
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   - -------------------------------------- 0.1/1.4 MB 825.8 kB/s eta 0:00:02
   --- ------------------------------------ 0.1/1.4 MB 1.2 MB/s eta 0:00:02
   ------- -------------------------------- 0.3/1.4 MB 1.7 MB/s eta 0:00:01
   --------- ------------------------------ 0.4/1.4 MB 1.6 MB/s eta 0:00:01
   --------------- ------------------------ 0.6/1.4 MB 2.2 MB/s eta 0:00:01
   -------------------- ------------------- 0.8/1.4 MB 2.5 MB/s eta 0:00:01
   ----------------------------- ---------- 1.1/1.4 MB 3.0 MB/s eta 0:00:01
   ------------------------------------- -- 1.4/1.4 MB 3.2 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 3.2 MB/s eta 0:00:00
Using cached scikit_learn-1.3.2-cp38-cp38-win_amd64.whl (9.3 MB)
  Attempting uninstall: scikit-learn

In [3]:
import pandas as pd
import numpy as np

#Data Visualization
import matplotlib.pyplot as plt

#Text Color
from termcolor import colored

#Train Test Split
from sklearn.model_selection import train_test_split

#Model Evaluation
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score
from mlxtend.plotting import plot_confusion_matrix

#Deep Learning
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GlobalMaxPooling1D, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import plot_model

In [4]:
import wandb
from wandb.keras import WandbCallback

# Login to wandb
wandb.login (key="8247272aa95730700b26a36fe0fe272034c54c4e")

wandb: Currently logged in as: nguyendatdtqn (nguyen_dat2). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\nguye/.netrc


True

## Data Preprocessing

In [5]:
# wandb config
WANDB_CONFIG = {
     'competition': 'Vietnamese News Classification', 
              '_wandb_kernel': 'neuracort'
    }

In [6]:
df_=pd.read_csv(r"E:\NLP\Vietnamese-Text-Classification-master\data\test.csv")

In [8]:
with open(r'E:\NLP\Vietnamese-Text-Classification-master\data\vietnamese-stopwords.txt', 'r', encoding='utf-8') as file:
    stop_words = file.read().splitlines()
print(stop_words)
df_['Contents'] = df_['Contents'].apply(lambda x: '' if isinstance(x, float) else x)
df_['Contents'] = df_['Contents'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))

['a lô', 'a ha', 'ai', 'ai ai', 'ai nấy', 'ai đó', 'alô', 'amen', 'anh', 'anh ấy', 'ba', 'ba ba', 'ba bản', 'ba cùng', 'ba họ', 'ba ngày', 'ba ngôi', 'ba tăng', 'bao giờ', 'bao lâu', 'bao nhiêu', 'bao nả', 'bay biến', 'biết', 'biết bao', 'biết bao nhiêu', 'biết chắc', 'biết chừng nào', 'biết mình', 'biết mấy', 'biết thế', 'biết trước', 'biết việc', 'biết đâu', 'biết đâu chừng', 'biết đâu đấy', 'biết được', 'buổi', 'buổi làm', 'buổi mới', 'buổi ngày', 'buổi sớm', 'bà', 'bà ấy', 'bài', 'bài bác', 'bài bỏ', 'bài cái', 'bác', 'bán', 'bán cấp', 'bán dạ', 'bán thế', 'bây bẩy', 'bây chừ', 'bây giờ', 'bây nhiêu', 'bèn', 'béng', 'bên', 'bên bị', 'bên có', 'bên cạnh', 'bông', 'bước', 'bước khỏi', 'bước tới', 'bước đi', 'bạn', 'bản', 'bản bộ', 'bản riêng', 'bản thân', 'bản ý', 'bất chợt', 'bất cứ', 'bất giác', 'bất kì', 'bất kể', 'bất kỳ', 'bất luận', 'bất ngờ', 'bất nhược', 'bất quá', 'bất quá chỉ', 'bất thình lình', 'bất tử', 'bất đồ', 'bấy', 'bấy chầy', 'bấy chừ', 'bấy giờ', 'bấy lâu', 'bấy lâ

In [9]:
drop_columns=['Date','Author(s)','Tags','URL']
train_data = df_.drop(columns=drop_columns)
train_data.head(10)

,Title,Summary,Contents,Category
0,"Hà Nội hỗ trợ hơn 4,5 tỉ đồng cho các tỉnh phí...","Trước tình hình mưa lớn gây lũ lụt, lũ ống, lũ...","đó, ảnh hưởng tỉnh Sơn La, Yên Bái, tỉnh Lai C...",Thời sự
1,Kiên Giang: Xúc tiến thương mại qua đường hàng...,"Kiên Giang - Chiều ngày 14.1, UBND tỉnh tổ chứ...","Thông hội nghị biết, 2019, tổng lượt du lịch K...",Kinh doanh
2,Xem trực tiếp TP.HCM vs Sài Gòn vòng 4 V.Leagu...,Báo Lao Động xin gửi tới quý độc giả link xem ...,"lượt trận vòng 4, bảng xếp hạng V.League 2020 ...",Thể thao
3,Những lời chúc ngày Thầy thuốc Việt Nam hay nhất,Những lời chúc ngày 27.2 hay và ý nghĩa để dàn...,"27.2.1955, Chủ tịch Hồ Chí Minh đích thân viết...",Sức khỏe
4,TPHCM: Đề nghị xử lý nghiêm vụ trật tự đô thị ...,UBND quận Bình Tân (TPHCM) đã ra văn bản đề ng...,"Chiều 11.5, trao đổi PV Lao Động, Nguyễn Gia T...",Pháp luật
5,"Chiến sĩ PCCC nói dối để cứu người, nữ thực tậ...",Chiến sĩ PCCC đã phải nói dối để trấn an người...,"đó, thực tập sinh Phan Thị Hiên (21 tuổi, sinh...",Diễn đàn
6,"Những chiêu trò lừa đảo, chiếm đoạt tiền phổ b...",Lợi dụng thời điểm cận Tết người dân có nhiều ...,Thủ đoạn cắp thông ngân hàng Lê Hoàng Minh – T...,Xã hội
7,Tottenham vs Dortmund: Chờ đợi màn đôi công má...,"Vào lúc 3h rạng sáng 14.2, Tottenham sẽ tiếp đ...","Tottenham phong độ đây, 3 trận Premier League ...",Thể thao
8,Lãnh đạo lâm trường bị đối tượng vi phạm dùng ...,"Ngày 17.10, Lâm trường Đồng Hới (Quảng Bình) c...","đó, 9h30 16.10, thực hiện nhiệm vụ tuần tra bả...",Pháp luật
9,"Google và Amazon giải quyết mâu thuẫn, đưa You...",Amazon (NASDAQ: AMZN) và Google (NASDAQ: GOOGL...,Google YouTube trở thiết Fire TV Amazon 2019. ...,Kinh doanh


In [10]:
def display_missing(df,features_columns):
    n_rows=df.shape[0]
    for col in features_columns:
        missing_count=df[col].isnull().sum()
        if missing_count>0:
            print(f"{col} has {missing_count} missing values")
column_names = train_data.columns
print("Train:",display_missing(train_data,column_names))

Summary has 5 missing values
Train: None


In [11]:
train_data = train_data.dropna(subset=['Contents'])
train_data = train_data.dropna(subset=['Summary'])

In [12]:
num_unique_classes = train_data['Category'].value_counts()

print("Số lượng lớp duy nhất trong cột là:", num_unique_classes)

class_counts = train_data['Category'].value_counts()

# Tạo danh sách các lớp có số lượng ít hơn 100
classes_to_remove = class_counts[class_counts < 100].index.tolist()

# Loại bỏ các dòng trong DataFrame có Category thuộc danh sách loại bỏ
train_data = train_data[~train_data['Category'].isin(classes_to_remove)]

Số lượng lớp duy nhất trong cột là: Xã hội                 897
Thể thao               527
Thế giới               485
Pháp luật              484
Kinh doanh             478
Công đoàn              395
Giáo dục               368
Văn hóa - Giải trí     293
Thời sự                291
Sức khỏe               192
Bạn đọc                135
Media                  116
Bất động sản           116
Xe +                    63
Gia đình - Hôn nhân     62
Tấm Lòng Vàng           31
Công nghệ               27
Lao Động cuối tuần       8
Tin tức việc làm         6
Lưu trữ                  5
Lao Động & Đời sống      5
Diễn đàn                 3
Tin hoạt động            3
Tin địa phương           2
Quỹ TLV                  1
Thông tin tiện ích       1
Sự kiện Bình luận        1
Name: Category, dtype: int64


In [13]:
num_unique_classes = train_data['Category'].value_counts()

print("Số lượng lớp duy nhất trong cột là:", num_unique_classes)

Số lượng lớp duy nhất trong cột là: Xã hội                897
Thể thao              527
Thế giới              485
Pháp luật             484
Kinh doanh            478
Công đoàn             395
Giáo dục              368
Văn hóa - Giải trí    293
Thời sự               291
Sức khỏe              192
Bạn đọc               135
Media                 116
Bất động sản          116
Name: Category, dtype: int64


In [14]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
train_data['Category'] = label_encoder.fit_transform(train_data['Category'])

In [15]:
from sklearn.model_selection import train_test_split
test_data = train_data.sample(n=3000, random_state=42)

train_data = train_data.drop(test_data.index)

# Lưu tập dữ liệu huấn luyện và kiểm tra thành các file CSV
train_data.to_csv(r'train_data.csv', index=False)
test_data.to_csv(r'test_data.csv', index=False)

In [16]:
TRAIN_FILE_PATH = r'train_data.csv'
TEST_FILE_PATH = r'test_data.csv'

#Load Data
data = pd.read_csv(TRAIN_FILE_PATH)
testdata = pd.read_csv(TEST_FILE_PATH)

In [17]:
data.head(10)

,Title,Summary,Contents,Category
0,Kiên Giang: Xúc tiến thương mại qua đường hàng...,"Kiên Giang - Chiều ngày 14.1, UBND tỉnh tổ chứ...","Thông hội nghị biết, 2019, tổng lượt du lịch K...",4
1,Xem trực tiếp TP.HCM vs Sài Gòn vòng 4 V.Leagu...,Báo Lao Động xin gửi tới quý độc giả link xem ...,"lượt trận vòng 4, bảng xếp hạng V.League 2020 ...",9
2,Những lời chúc ngày Thầy thuốc Việt Nam hay nhất,Những lời chúc ngày 27.2 hay và ý nghĩa để dàn...,"27.2.1955, Chủ tịch Hồ Chí Minh đích thân viết...",7
3,TPHCM: Đề nghị xử lý nghiêm vụ trật tự đô thị ...,UBND quận Bình Tân (TPHCM) đã ra văn bản đề ng...,"Chiều 11.5, trao đổi PV Lao Động, Nguyễn Gia T...",6
4,"Những chiêu trò lừa đảo, chiếm đoạt tiền phổ b...",Lợi dụng thời điểm cận Tết người dân có nhiều ...,Thủ đoạn cắp thông ngân hàng Lê Hoàng Minh – T...,12
5,Vườn hoa ở vùng mỏ sắt Thạch Khê thành điểm 's...,"Những ngày nghỉ Tết Tân Sửu, hàng trăm người đ...","Chiều 16.2 (mùng 5 Tết), dân khắp xa, đặc biệt...",11
6,Tình báo Israel được điều động theo dõi biến t...,Israel là nước đầu tiên trên thế giới điều độn...,phủ Israel thị quan tình báo nội SHABAK dõi tu...,8
7,"Đơn vị thành viên Tổng Công ty Rau quả, nông s...",Công đoàn Nông nghiệp và phát triển nông thôn ...,4 Công đoàn sở gồm: Công đoàn Công ty Cổ In Ba...,5
8,"Cắt giảm điều kiện kinh doanh trước ""giờ G"" 30...",Thời hạn cắt giảm điều kiện kinh doanh đặt ra ...,"đó, “Không phiên họp Thủ tướng nhắc, văn đôn đ...",4
9,"Hiểu đúng vai trò, trách nhiệm của chủ đầu tư ...",Chủ đầu tư có vai trò vô cùng quan trọng đối v...,Vai trò chủ đầu tư Chủ đầu tư khả năng tổ chức...,1


In [18]:
drop_columns=['Title','Summary']
data = data.drop(columns=drop_columns)
testdata = testdata.drop(columns=drop_columns)

In [19]:
#Set Column Names 
data.columns = ['Contents','Category']
testdata.columns = ['Contents','Category']

#Combine Title and Description
X_train = data['Contents'] 
y_train = data['Category'].apply(lambda x: x-1).values # Class labels need to begin from 0

x_test = testdata['Contents'] # Combine title and description (better accuracy than using them as separate features)
y_test = testdata['Category'].apply(lambda x: x-1).values # Class labels need to begin from 0

In [20]:
X_train

0       Thông hội nghị biết, 2019, tổng lượt du lịch K...
1       lượt trận vòng 4, bảng xếp hạng V.League 2020 ...
2       27.2.1955, Chủ tịch Hồ Chí Minh đích thân viết...
3       Chiều 11.5, trao đổi PV Lao Động, Nguyễn Gia T...
4       Thủ đoạn cắp thông ngân hàng Lê Hoàng Minh – T...
                              ...                        
1772    25.3, Công an tỉnh Bình Dương thông tạm nghi c...
1773    Tuyển Malaysia buộc thắng Indonesia trận đấu t...
1774    19h45 tối (5.6), đội tuyển Việt Nam trận quân ...
1775    hội nghị, Nguyễn Minh Dũng - Phó Trưởng Ban Tà...
1776    24.5, Công an huyện Hương Sơn (Hà Tĩnh) biết, ...
Name: Contents, Length: 1777, dtype: object

In [21]:
# Loại bỏ các giá trị NaN khỏi X_train trước khi tính toán chiều dài tối đa
X_train_cleaned = X_train.dropna()

# Tính toán chiều dài tối đa của chuỗi trong X_train sau khi loại bỏ NaN
maxlen = X_train_cleaned.map(lambda x: len(x.split())).max()

# In ra chiều dài tối đa
print("Chiều dài tối đa của chuỗi trong X_train:", maxlen)

Chiều dài tối đa của chuỗi trong X_train: 5172


## Exploratory Data Analysis

### Data Shape

In [22]:
data.shape, testdata.shape

((1777, 2), (3000, 2))

### Value Counts

In [23]:
#Checking Value counts to determine class balance
data.Category.value_counts()

12    335
9     208
8     203
4     164
6     154
2     143
3     138
10    107
11     94
7      76
0      57
5      50
1      48
Name: Category, dtype: int64

In [24]:
testdata.Category.value_counts()

12    562
6     330
9     319
4     314
8     282
2     252
3     230
11    199
10    184
7     116
0      78
1      68
5      66
Name: Category, dtype: int64

### Null Values

In [25]:
# #Train Data
data.isnull().sum()

Contents    23
Category     0
dtype: int64

In [26]:
# #Test Data
testdata.isnull().sum()

Contents    56
Category     0
dtype: int64

## Tokenize and Pad Data

### Tokenization

Tokenization is a common task in Natural Language Processing (NLP). It’s a fundamental step in both traditional NLP methods like Count Vectorizer and Advanced Deep Learning-based architectures like Transformers.

Tokenization is a way of separating a piece of text into smaller units called tokens. Here, tokens can be either words, characters, or subwords. Hence, tokenization can be broadly classified into 3 types – word, character, and subword (n-gram characters) tokenization.

For example, consider the sentence: “Never give up”.

The most common way of forming tokens is based on space. Assuming space as a delimiter, the tokenization of the sentence results in 3 tokens – Never-give-up. As each token is a word, it becomes an example of Word tokenization.

Similarly, tokens can be either characters or subwords. For example, let us consider “smarter”:

1. Character tokens: s-m-a-r-t-e-r
2. Subword tokens: smart-er

### Padding
All the neural networks require to have inputs that have the same shape and size. However, when we pre-process and use the texts as inputs for our model e.g. LSTM, not all the sentences have the same length. In other words, naturally, some of the sentences are longer or shorter. We need to have the inputs with the same size, this is where the padding is necessary.

In [27]:
vocab_size = 5000 # arbitrarily chosen
embed_size = 32 # arbitrarily chosen
maxlen=5000
X_train = X_train.astype(str)
x_test=x_test.astype(str)
# Create and Fit tokenizer
tok = Tokenizer(num_words=vocab_size)
tok.fit_on_texts(X_train.values)

# Tokenize data
X_train = tok.texts_to_sequences(X_train)
x_test = tok.texts_to_sequences(x_test)

# Pad data
X_train = pad_sequences(X_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [28]:
X_train

array([[   0,    0,    0, ...,   27,  108,   86],
       [   0,    0,    0, ..., 1084,  857,  908],
       [   0,    0,    0, ...,  478,   36,   15],
       ...,
       [   0,    0,    0, ...,  902,  606,  553],
       [   0,    0,    0, ...,   40,   14,  581],
       [   0,    0,    0, ...,   38,  216,   46]])

## Model

### LSTM - 
To solve the problem of Vanishing and Exploding Gradients in a deep Recurrent Neural Network, many variations were developed. One of the most famous of them is the Long Short Term Memory Network(LSTM). In concept, an LSTM recurrent unit tries to “remember” all the past knowledge that the network is seen so far and to “forget” irrelevant data. This is done by introducing different activation function layers called “gates” for different purposes. Each LSTM recurrent unit also maintains a vector called the Internal Cell State which conceptually describes the information that was chosen to be retained by the previous LSTM recurrent unit. A Long Short Term Memory Network consists of four different gates for different purposes as described below:-

1. **Forget Gate(f)**: It determines to what extent to forget the previous data.

2. **Input Gate(i)**: It determines the extent of information to be written onto the Internal Cell State.

3. **Input Modulation Gate(g**): It is often considered as a sub-part of the input gate and many literatures on LSTM’s do not even mention it and assume it inside the Input gate. It is used to modulate the information that the Input gate will write onto the Internal State Cell by adding non-linearity to the information and making the information Zero-mean. This is done to reduce the learning time as Zero-mean input has faster convergence. Although this gate’s actions are less important than the others and is often treated as a finesse-providing concept, it is good practice to include this gate into the structure of the LSTM unit.

4. **Output Gate(o)**: It determines what output(next Hidden State) to generate from the current Internal Cell State.

The basic work-flow of a Long Short Term Memory Network is similar to the work-flow of a Recurrent Neural Network with only difference being that the Internal Cell State is also passed forward along with the Hidden State.

### BiDirectional LSTM - 
Using bidirectional will run our inputs in two ways, one from past to future and one from future to past and what differs this approach from unidirectional is that in the LSTM that runs backwards we preserve information from the future and using the two hidden states combined we are able in any point in time to preserve information from both past and future.

In [29]:
run = wandb.init(project='Vietnamese News Classification', config= WANDB_CONFIG)

In [30]:
model = Sequential()
model.add(Embedding(vocab_size, embed_size, input_length=maxlen))
model.add(Bidirectional(LSTM(128, return_sequences=True))) 
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(GlobalMaxPooling1D()) #Pooling Layer decreases sensitivity to features, thereby creating more generalised data for better test results.
model.add(Dense(1024))
model.add(Dropout(0.25)) #Dropout layer nullifies certain random input values to generate a more general dataset and prevent the problem of overfitting.
model.add(Dense(512))
model.add(Dropout(0.25))
model.add(Dense(256))
model.add(Dropout(0.25))
model.add(Dense(128))
model.add(Dropout(0.25))
model.add(Dense(64))
model.add(Dropout(0.25))
model.add(Dense(23, activation='softmax')) #softmax is used as the activation function for multi-class classification problems where class membership is required on more than two class labels.
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 5000, 32)          160000    
                                                                 
 bidirectional (Bidirection  (None, 5000, 256)         164864    
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 5000, 128)         164352    
 onal)                                                           
                                                                 
 global_max_pooling1d (Glob  (None, 128)               0         
 alMaxPooling1D)                                                 
                                                                 
 dense (Dense)               (None, 1024)              132096    
                                                        

## Compile and Fit Model

In [31]:
callbacks = [
    EarlyStopping(     #EarlyStopping is used to stop at the epoch where val_accuracy does not improve significantly
        monitor='val_accuracy',
        min_delta=1e-4,
        patience=4,
        verbose=1
    ),
    ModelCheckpoint(
        filepath='weights.h5',
        monitor='val_accuracy', 
        mode='max', 
        save_best_only=True,
        save_weights_only=True,
        verbose=1
    ),
    WandbCallback()
]

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


In [ ]:
#Compile and Fit Model

model.compile(loss='sparse_categorical_crossentropy', #Sparse Categorical Crossentropy Loss because data is not one-hot encoded
              optimizer='adam', 
              metrics=['accuracy']) 

model.fit(X_train, 
          y_train, 
          batch_size=15000, 
          validation_data=(x_test, y_test), 
          epochs=30, 
          callbacks=callbacks)
keras.backend.clear_session()
gc.collect()

# Close W&B run
wandb.finish()

Epoch 1/30


## Load Weights with Best val Accuracy

In [ ]:
#model.load_weights('weights.h5')
model.save(r'/kaggle/working/model.hdf5')

## Model Demo

In [ ]:
# def modelDemo(news_text):

#   #News Labels
#   labels = ['World News', 'Sports News', 'Business News', 'Science-Technology News']

#   test_seq = pad_sequences(tok.texts_to_sequences(news_text), maxlen=maxlen)

#   test_preds = [labels[np.argmax(i)] for i in model.predict(test_seq)]

#   for news, label in zip(news_text, test_preds):
#       # print('{} - {}'.format(news, label))
#       print('{} - {}'.format(colored(news, 'yellow'), colored(label, 'blue')))


In [ ]:
# modelDemo(['New evidence of virus risks from wildlife trade'])

In [ ]:
# modelDemo(['Coronavirus: Bank pumps £100bn into UK economy to aid recovery'])

In [ ]:
# modelDemo(['Trump\'s bid to end Obama-era immigration policy ruled unlawful'])

In [ ]:
# modelDemo(['David Luiz’s future with Arsenal to be decided this week'])

In [ ]:
# modelDemo(['Indian Economic budget supports the underprivileged sections of society'])

## Model Evaluation

### Confusion Matrix

In [ ]:
# labels = ['World News', 'Sports News', 'Business News', 'Science-Technology News']
# preds = [np.argmax(i) for i in model.predict(x_test)]
# cm  = confusion_matrix(y_test, preds)
# plt.figure()
# plot_confusion_matrix(cm, figsize=(16,12), hide_ticks=True, cmap=plt.cm.Blues)
# plt.xticks(range(4), labels, fontsize=12)
# plt.yticks(range(4), labels, fontsize=12)
# plt.show()

### Recall, Precision and Accuracy

In [ ]:
# print("Recall of the model is {:.2f}".format(recall_score(y_test, preds, average='micro')))
# print("Precision of the model is {:.2f}".format(precision_score(y_test, preds, average='micro')))
# print("Accuracy of the model is {:.2f}".format(accuracy_score(y_test, preds)))